In [7]:
import os
import torch
from PIL import Image
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from sklearn.metrics import classification_report, confusion_matrix

# Load the model and feature extractor
extractor = AutoFeatureExtractor.from_pretrained("rafalosa/diabetic-retinopathy-224-procnorm-vit")
model = AutoModelForImageClassification.from_pretrained("rafalosa/diabetic-retinopathy-224-procnorm-vit")

c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [15]:
# Evaluation data folder path and mapping of folder names to labels
evaluation_data_folder = "images/evaluation2/"
label_mapping = {
    "No_DR": 0,
    "Mild": 1,
    "Moderate": 2,
    "Severe": 3,
    "Proliferate_DR": 4
}

# Reverse the label_mapping to get the original names from numbers
reverse_label_mapping = {v: k for k, v in label_mapping.items()}


In [18]:
# Loop function to print images with their folder names as labels and model outputs
def print_images_with_labels_and_model_outputs(data_folder, label_mapping):
    model.eval()

    for folder_name in os.listdir(data_folder):
        folder_path = os.path.join(data_folder, folder_name)
        label = reverse_label_mapping.get(label_mapping.get(folder_name, -1), "Unknown Label")

        print(f"Images in folder '{folder_name}' (Label: {label}):")
        for image_filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_filename)
            image = Image.open(image_path)
            inputs = extractor(images=image, return_tensors="pt")
            outputs = model(**inputs)
            logits = outputs.logits
            probs = logits.softmax(dim=1)

            # Get all labels and scores for the image
            predictions = [{'score': score.item(), 'label': reverse_label_mapping.get(i, "Unknown Label")}
                           for i, score in enumerate(probs[0])]

            # Sort the predictions by scores in descending order
            predictions.sort(key=lambda x: x['score'], reverse=True)

            print(f"Image: {image_path}")
            print(f"True Label: {label}")

            # Print all scores and labels
            for pred in predictions:
                print(f"Label: {pred['label']}, Score: {pred['score']:.4f}")

            # Get the label with the highest score as the predicted label
            predicted_label = predictions[0]['label']
            confidence_score = predictions[0]['score']

            print(f"Predicted Label: {predicted_label}, Confidence: {confidence_score:.4f}")
            print("=" * 40)

# Call the loop function
print_images_with_labels_and_model_outputs(evaluation_data_folder, label_mapping)

Images in folder 'Mild' (Label: Mild):
Image: images\evaluation2\Mild\172df1330a60.png
True Label: Mild
Label: Mild, Score: 0.4472
Label: Proliferate_DR, Score: 0.1659
Label: Moderate, Score: 0.1627
Label: Severe, Score: 0.1259
Label: No_DR, Score: 0.0984
Predicted Label: Mild, Confidence: 0.4472
Image: images\evaluation2\Mild\1a03a7970337.png
True Label: Mild
Label: Mild, Score: 0.4824
Label: Moderate, Score: 0.3330
Label: No_DR, Score: 0.0787
Label: Proliferate_DR, Score: 0.0572
Label: Severe, Score: 0.0487
Predicted Label: Mild, Confidence: 0.4824
Image: images\evaluation2\Mild\1b329a127307.png
True Label: Mild
Label: Mild, Score: 0.5808
Label: Moderate, Score: 0.1606
Label: Proliferate_DR, Score: 0.1110
Label: No_DR, Score: 0.0840
Label: Severe, Score: 0.0635
Predicted Label: Mild, Confidence: 0.5808
Image: images\evaluation2\Mild\1bb0ddfe753a.png
True Label: Mild
Label: Mild, Score: 0.4671
Label: Moderate, Score: 0.1670
Label: Proliferate_DR, Score: 0.1517
Label: Severe, Score: 0.

In [17]:

# Function to generate classification report and confusion matrix for a given folder
def generate_metrics_for_folder(data_folder, label_mapping):
    true_labels = []
    predicted_labels = []

    for image_filename in os.listdir(data_folder):
        image_path = os.path.join(data_folder, image_filename)
        image = Image.open(image_path)
        inputs = extractor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        probs = logits.softmax(dim=1)
        predicted_class_idx = torch.argmax(probs, dim=1).item()
        predicted_label = reverse_label_mapping.get(predicted_class_idx, "Unknown Label")

        folder_name = os.path.basename(data_folder)
        label = reverse_label_mapping.get(label_mapping.get(folder_name, -1), "Unknown Label")

        true_labels.append(label)
        predicted_labels.append(predicted_label)

    print(f"Folder: '{folder_name}'")
    print("Classification Report:")
    print(classification_report(true_labels, predicted_labels))

    print("Confusion Matrix:")
    print(confusion_matrix(true_labels, predicted_labels))
    print("=" * 40)

# Call the function for each folder to generate metrics per folder
for folder_name in os.listdir(evaluation_data_folder):
    folder_path = os.path.join(evaluation_data_folder, folder_name)
    generate_metrics_for_folder(folder_path, label_mapping)

Folder: 'Mild'
Classification Report:
              precision    recall  f1-score   support

        Mild       1.00      0.73      0.85        30
    Moderate       0.00      0.00      0.00         0

    accuracy                           0.73        30
   macro avg       0.50      0.37      0.42        30
weighted avg       1.00      0.73      0.85        30

Confusion Matrix:
[[22  8]
 [ 0  0]]


c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

Folder: 'Moderate'
Classification Report:
              precision    recall  f1-score   support

        Mild       0.00      0.00      0.00         0
    Moderate       1.00      0.10      0.18        30

    accuracy                           0.10        30
   macro avg       0.50      0.05      0.09        30
weighted avg       1.00      0.10      0.18        30

Confusion Matrix:
[[ 0  0]
 [27  3]]


c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

Folder: 'No_DR'
Classification Report:
              precision    recall  f1-score   support

    Moderate       0.00      0.00      0.00       0.0
       No_DR       0.00      0.00      0.00      30.0

    accuracy                           0.00      30.0
   macro avg       0.00      0.00      0.00      30.0
weighted avg       0.00      0.00      0.00      30.0

Confusion Matrix:
[[ 0  0]
 [30  0]]


c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

Folder: 'Proliferate_DR'
Classification Report:
                precision    recall  f1-score   support

          Mild       0.00      0.00      0.00       0.0
      Moderate       0.00      0.00      0.00       0.0
Proliferate_DR       0.00      0.00      0.00      30.0

      accuracy                           0.00      30.0
     macro avg       0.00      0.00      0.00      30.0
  weighted avg       0.00      0.00      0.00      30.0

Confusion Matrix:
[[ 0  0  0]
 [ 0  0  0]
 [28  2  0]]


c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

Folder: 'Severe'
Classification Report:
              precision    recall  f1-score   support

        Mild       0.00      0.00      0.00       0.0
    Moderate       0.00      0.00      0.00       0.0
      Severe       0.00      0.00      0.00      30.0

    accuracy                           0.00      30.0
   macro avg       0.00      0.00      0.00      30.0
weighted avg       0.00      0.00      0.00      30.0

Confusion Matrix:
[[ 0  0  0]
 [ 0  0  0]
 [29  1  0]]


c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bryon\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie